# 1. Library Import

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = './data/'

# 2. 학습 데이터 생성

## 2.1  train_err

- train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.
- 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.

In [2]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [3]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [4]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [5]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:30<00:00, 550883.94it/s]


(15000, 42)

In [6]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [7]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [8]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 42)
(15000,)


In [32]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      { 'learning_rate' : 0.003,
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'num_leaves'    : '31',
                'max_depth'     : '-1',
                'feature_fraction': 0.7,
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20,
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001709 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4353
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
[20]	valid_0's auc: 0.788249	valid_0's pr_auc: 0.779763
[40]	valid_0's auc: 0.789743	valid_0's pr_auc: 0.781685
[60]	valid_0's auc: 0.790943	valid_0's pr_auc: 0.782311
[80]	valid_0's auc: 0.790989	valid_0's pr_auc: 0.782492
[100]	valid_0's auc: 0.791435	valid_0's pr_auc: 0.783292
[120]	valid_0's auc: 0.791726	valid_0's pr_auc: 0.784031
[140]	valid_0's auc: 0.792267	valid_0's pr_auc: 0.7846
[160]	valid_0's auc: 0.792864	valid_0's pr_auc: 0.785325
[180]	valid_0's auc: 0.792919	valid_0's pr_auc: 0.78552

[360]	valid_0's auc: 0.77991	valid_0's pr_auc: 0.380736
[380]	valid_0's auc: 0.780112	valid_0's pr_auc: 0.382176
[400]	valid_0's auc: 0.780311	valid_0's pr_auc: 0.382701
[420]	valid_0's auc: 0.780432	valid_0's pr_auc: 0.383449
[440]	valid_0's auc: 0.780747	valid_0's pr_auc: 0.38555
[460]	valid_0's auc: 0.781263	valid_0's pr_auc: 0.387022
[480]	valid_0's auc: 0.781576	valid_0's pr_auc: 0.389532
[500]	valid_0's auc: 0.782054	valid_0's pr_auc: 0.390471
[520]	valid_0's auc: 0.782493	valid_0's pr_auc: 0.390046
[540]	valid_0's auc: 0.782814	valid_0's pr_auc: 0.390934
[560]	valid_0's auc: 0.783167	valid_0's pr_auc: 0.39178
[580]	valid_0's auc: 0.783514	valid_0's pr_auc: 0.392842
[600]	valid_0's auc: 0.783721	valid_0's pr_auc: 0.393259
[620]	valid_0's auc: 0.784079	valid_0's pr_auc: 0.394106
[640]	valid_0's auc: 0.784395	valid_0's pr_auc: 0.393628
[660]	valid_0's auc: 0.784593	valid_0's pr_auc: 0.39331
[680]	valid_0's auc: 0.784723	valid_0's pr_auc: 0.393001
[700]	valid_0's auc: 0.784843	valid

[940]	valid_0's auc: 0.812321	valid_0's pr_auc: 0.975064
[960]	valid_0's auc: 0.812513	valid_0's pr_auc: 0.975091
[980]	valid_0's auc: 0.812586	valid_0's pr_auc: 0.975091
[1000]	valid_0's auc: 0.812726	valid_0's pr_auc: 0.975141


# 4. 교차검증 점수 확인

In [33]:
print(np.mean(auc_scores))

0.8057416626886568


# 5. 제출 파일 생성

In [34]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [35]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [36]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:29<00:00, 557383.53it/s]

(14999, 42)


In [37]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [38]:
pred_ensemble

array([[0.80943472],
       [0.21131445],
       [0.30199256],
       ...,
       [0.46700933],
       [0.84082397],
       [0.35958797]])

In [39]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [40]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [41]:
sample_submssion.to_csv("dacon_baseline.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.809435
1,30001,0.211314
2,30002,0.301993
3,30003,0.677297
4,30004,0.606094
...,...,...
14994,44994,0.322464
14995,44995,0.326018
14996,44996,0.467009
14997,44997,0.840824
